In [45]:
import pandas as pd
import numpy as np

### Бутстрап

Задача: магазин хочет склонить покупателей к скачиванию мобильного приложения. В качестве эксперимента форму для получения ссылки заменили на кнопку, ведущую в AppStore. На серверах с ID 2 и 3 осталась контрольная версия с формой, на сервере с ID 1 — версия с кнопкой. Проверить, есть ли значимые изменения в средней доле переходов к скачиванию (`VisitPageFlag`)

In [46]:
grocery_data = pd.read_csv("../data/grocery_website.csv")
grocery_data.drop(columns=['LoggedInFlag'])
grocery_data

,RecordID,IP Address,LoggedInFlag,ServerID,VisitPageFlag
0,1,39.13.114.2,1,2,0
1,2,13.3.25.8,1,1,0
2,3,247.8.211.8,1,1,0
3,4,124.8.220.3,0,3,0
4,5,60.10.192.7,0,2,0
...,...,...,...,...,...
184583,184584,114.8.104.1,0,1,0
184584,184585,207.2.110.5,0,2,1
184585,184586,170.13.31.9,0,2,0
184586,184587,195.14.92.3,0,3,0


Некоторые пользователи (идентифицируем их по IP-адресу) заходили на сайт несколько раз, удалите дубликаты пользователей и для каждого заполните `VisitPageFlag=1`, если хотя бы раз для него встречалось значение 1

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Метрика: отношение количества посещений сайта с `VisitPageFlag=1` к общему количеству посещений.<br>
Для обеих групп сгенерируйте бутстрапированное распределение разностей значения метрик для групп A и B.

In [ ]:
iterations = 3000
A = grocery_data[grocery_data.ServerID > 1]
B = grocery_data[grocery_data.ServerID == 1]

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Хоть полученное среднее метрик и близко к значению метрики на генеральной совокупности, оно всё равно будет немного смещено. Это легко исправить. Научимся делать коррекцию смещения (bias correction):
1. Найдите среднее полученных бутстрапом значений, назовём его $\Theta_{boot}$
2. Найдите разность метрик для A и B ГС: $\Theta_{orig}$
3. $|\Theta_{orig} - \Theta_{boot}|$ и есть модуль смещения
4. Сдвиньте все полученные бутстрапом значения на эту величину

Не перепутайте, в какую сторону сдвигать. Проверьте результат, выведя $\Theta_{orig}$, $\Theta_{boot}$ и среднее скорректированных значений.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Далее используйте только скорректированные значения. Найдите границы доверительного интервала и проверьте, значима ли разница значений метрик для A и B (находится ли 0 за пределами интервала)

In [ ]:
alpha = 0.05

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Вычислите среднее значение скорректированной разности, нам понадобится его запомнить:

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### Бакетизация

Бутстрапирование на большом объёме данных может происходить очень долго.<br>
* Присвойте каждой записи один из `buckets_count` индексов
* Сгруппируйте в бакеты записи с одинаковыми индексами и группами эксперимента (A/B)
* Для каждого бакета посчитайте сумму `VisitPageFlag` и количество записей

In [ ]:
buckets_count = 5000

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Теперь на полученных выборках повторите процедуру бутстрапа.

In [ ]:
iterations = 3000

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Скорректируйте значения, возьмите среднее и сравните с результатом до бакетизации.<br>

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Какой можно сделать вывод? Подтверждена или опровергнута $H_0$?

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### Дельта-метод для ratio-метрик

Потренируйтесь использовать дельта-метод на задаче из прошлой темы. Для упрощения будем смотреть на неё как на A/B. Вспомните, на какие два типа событий мы смотрели.

In [10]:
logs_data = pd.read_csv("../data/logs_exp_us.csv", sep='\t')
logs_data = logs_data.drop_duplicates()
logs_data['ExpId'] = logs_data['ExpId'].replace({246: 'A', 247: 'A', 248: 'B'})
logs_data = logs_data.rename(columns={'ExpId': 'TestGroup'})
logs_data

,EventName,DeviceIDHash,EventTimestamp,TestGroup
0,MainScreenAppear,4575588528974610257,1564029816,A
1,MainScreenAppear,7416695313311560658,1564053102,A
2,PaymentScreenSuccessful,3518123091307005509,1564054127,B
3,CartScreenAppear,3518123091307005509,1564054127,B
4,PaymentScreenSuccessful,6217807653094995999,1564055322,B
...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,A
244122,MainScreenAppear,5849806612437486590,1565212439,A
244123,MainScreenAppear,5746969938801999050,1565212483,A
244124,MainScreenAppear,5746969938801999050,1565212498,A


Переведите таймстампы в даты, сгруппируйте по ним и посчитайте для каждого дня количество событий, чтобы события нужных типов мы использовали по аналогии с CTR в образце:

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Найдите ковариации для групп A и B:

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Дисперсии ratio-метрик:

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Наконец, найдите p-value для t-статистики:

In [ ]:
from scipy.stats import norm

### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Кажется ли вам полученное значение странным? Посмотрите, какого размера были выборки:

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

Похоже, тут нам поможет только бутстрап. А лучше продление эксперимента.